#  From Visualization to Statistics
----

## Limited utility of graphing vectors

Previously, we tried to analyze data using different kinds of visualizations. 
Graphs are easy to understand and catchy when you want to explain something. 
You can convey lot of information through a picture. 
However, as we saw, there is a limit to the extent of information you can show on the graph and still convey its meaning appropriately. 

We have seen graphs do a great job in univariate and bivariate data analysis. 
They did fine in explaining how data varies in four dimensions by plotting 3-dimensional data 
points and using color as a fourth feature. 
However, it is hard to plot 5 features within a single plot. 
Even if you manage to plot 5 dimensions it will be difficult to interpret the graph. 
So when you have multidimensional data, which can reach 1000's of features, 
you have to rely on multivariate statistics for analyzing the data before you explore visualizations.

In fact, statistics will help you to narrow down the list of significant features you can use for building a model. 
These reduced sets of features can be analyzed using graphs. 
Statistics will guide which features or data to be visualized to gain more insights from the data.

#### Let's review multidimensional visualizations from module 2 using shapes / colors / size

We will work with the Kings county housing data again...

In [ ]:
housing_prices <- read.csv("/dsa/data/all_datasets/house_sales_in_king_county/kc_house_data.csv")
head(housing_prices)

Let's see how the independent variables are correlated to the dependent variable price. 
Before we can run `cor()` command on **housing_prices**, we need to make sure all variables supplied to `cor()` function are numeric.

__Reference__: [apply, sapply, lapply in R](https://www.r-bloggers.com/using-apply-sapply-lapply-in-r/)  
__Reference__: [R function for determining Class, TypeOf](http://rfunction.com/archives/770)  
__Reference__: [R transpose function, t()](http://www.statmethods.net/management/reshape.html)

In [ ]:
# sapply applies the class function to each column of housing_prices dataframe.
t(sapply(housing_prices,class))
# Transpose the output to a row instead of a column

Notice that `date` is a factor variable. 
As such, we should exclude it when we use the `cor()` function on the housing_prices dataframe. 
Also, `id` is just a row identifier (aka counter)and is useless for statistical modelling. 
We will exclude both of them as shown below. 

In [ ]:
# Selecting columns from housing_prices which are not equal to id and date. 
cor(housing_prices[!names(housing_prices) %in% c('id','date')])

# Note this sub-selects the columns that are not in the list (id,date)
# The resulting data is sent to the correlation function to become a correlation matrix.

So, the remaining variables are... 
  1. `bathrooms`, 
  1. `sqft_living`, 
  1. `grade`, 
  1. `sqft_above` and 
  1. `sqft_living15`  
They look significant, since they have a correlation greater than 0.5.

How can you visualize these five features relative to price? 
Previously, we plotted each one against price and looked at the relationships in the value growth patterns.

In [ ]:
options(scipen=999)  
library(ggplot2)
ggplot(housing_prices,
       aes(x=sqft_living,        # independent variable, feature 1
           y=price,              # dependent variable, feature 2
           color=factor(grade),  # independent variable, feature 3
           size=bathrooms),      # independent variable, feature 4
       xlab("sqft living"),
       ylab("price"),
       main("Prive vs Sqft living area")
      ) +geom_point()

Houses with 0 bathrooms and 0 bedrooms are typical outliers. 
Let's remove those rows and redo the plot above. 

In [ ]:
housing_prices=housing_prices[!housing_prices$bedrooms %in% c(0) & !housing_prices$bathrooms %in% c(0),]

In [ ]:
ggplot(housing_prices,
       aes(x=sqft_living,
           y=price,
           color=factor(grade),
           size=bathrooms),
       xlab("sqft living"),
       ylab("price"),
       main("Prive vs Sqft living area")
      ) +geom_point() +geom_smooth(method='lm',color='black')

We have a nice visualization summarizing `price` against `sqft_living`, `grade` and `bathrooms`. 
Price is increasing linearly with increases in sqft_living. 
Big houses tend to have better grades. 

However, we are not able to interpret the size of the datapoints clearly for number of bathrooms. 
This is the limitation with visualizations. 
When the data becomes huge, plots become difficult to interpret.

Let's try to denote `bathrooms` with color and `sqft_above` with grade... 

In [ ]:
ggplot(housing_prices,
       aes(x=sqft_living,
           y=price,
           color=factor(bathrooms),
           size=sqft_above),
       xlab("sqft living"),ylab("price"),
       main("Prive vs Sqft living area")
      ) +geom_point() +geom_smooth(method='lm',color='black')

The plot does well summarizing `price` against `sqft_living` but explains little about how `price` is correlates with `sqft_above` or number of `bathrooms`. 
The size of the datapoints denote the sqft_above. 
Since the observations are closely packed we are not able to identify the trend in the data along with `sqft_living`. 
Bathrooms, when used as a factor, has 29 levels which are difficult to interpret with such subtle changes and the closely clustered data. 

We can try the same thing with a 3-dimensional plot to see if the size and color parameters help us in analyzing multiple dimensions in the data. 

__Reference__: [with() function](http://rfunction.com/archives/2182)  
__Reference__: [Scatter Plot 3D](https://cran.r-project.org/web/packages/scatterplot3d/scatterplot3d.pdf)

In [ ]:
library(scatterplot3d)
#Assigning a color to ranges of bathrooms
housing_prices$colors[housing_prices$bathrooms<2] <- "red"
housing_prices$colors[housing_prices$bathrooms==2] <- "blue"
housing_prices$colors[housing_prices$bathrooms>2] <- "green"
housing_prices$colors[housing_prices$bathrooms>3] <- "magenta"
housing_prices$colors[housing_prices$bathrooms>4] <- "cyan"


with(housing_prices, {
   # The scatter plot 3D was introduced in the extra / optional notebooks last week.
   # This function produces a three dimensional plot of points using 3 variables for position, instead of just 2.
                 #   x,        y,   and    z axis
   scatterplot3d(sqft_living, sqft_above, price,       
                 type="h",             # put lines on the horizontal plane
                 angle = 45,           # angle=45 denotes how graph is oriented,
                 pch = 16,             # pch=16 denotes shape used to denote points on the plot
                 color=housing_prices$colors,  
                           #color=colors tells the graph to use colors variables defined above
                 main="Areas vs Prices",        
                 xlab="Sqft living area",
                 ylab="Sqft above area",
                 zlab="Price")

legend("topleft", inset=.05,      # location where the legend should be positioned on the graph
    bty="n", cex=.5,              # suppress legend box, shrink text 50%
    title="Bathrooms", 
    c("<2", "2", ">2", ">3", ">4"), fill=c("red", "blue", "green", "magenta", "cyan"))
}) # ends the context of the with()

While we have been able to incorporate additional features of the data into the plot, we can see we have reached the limit in dimensionality of the data that can be visually inspected.

## Feature Selection

Up to this point, we have engaged in _feature selection_.  

**Feature Selection** is the process of choosing relevant features from a data set to perform modelling and machine learning tasks.   

We have chose features based on a simple threshold of 0.5 for correlation between an independent variable and our desired predicted variable.

In the next lab, you will learn about additional techniques to reduce the feature space.
